In [71]:
import numpy as np

In [72]:
labels = [b'NO', b'DH', b'SL']
data = np.loadtxt('column_3C.dat', converters={6: lambda s: labels.index(s)})

x = data[:,0:6]
y=data[:,6]

training_indices = list(range(0,20)) + list(range(40,188)) + list(range(230,310))
test_indices = list(range(20,40)) + list(range(188, 230))

trainx = x[training_indices,:]
trainy = y[training_indices]
testx = x[test_indices]
testy = y[test_indices]

print(np.shape(trainx))
print(len(trainy))

(248, 6)
248


In [73]:
#Straight From the NumpyDemo
def squared_dist(x_vect, y_vect):
    return(np.sum(np.square(x_vect - y_vect)))


squared_dist(trainx[0,],testx[0,])

1879.5791000000006

In [75]:
# Broke the NN_L2 function into individual pieces that I found in the demo
def fNN_L2(x):
    distances = [squared_dist(x, trainx[i]) for i in range(len(trainy))]
    return distances[np.argmin(distances)]

def NN_classifier(x):
    index=fNN_L2(x)
    return trainy[index]

print(fNN_L2(testx[5]))


65.8469


In [76]:

# Mushed them together
def NN_L2(xvect, yvect, zvect):
    predict=[]
    for z in zvect:
        distances = [squared_dist(z, xvect[i]) for i in range(len(yvect))]
        index=np.argmin(distances)
        predict.append(yvect[index])
    return np.array(predict)

testy_L2=NN_L2(trainx, trainy, testx)

print( type( testy_L2) )
print( len(testy_L2) )
print( testy_L2[40:50] )



<class 'numpy.ndarray'>
62
[2. 2. 1. 0. 0. 0. 0. 0. 0. 0.]


In [77]:
testy_L2 = NN_L2(trainx, trainy, testx)

assert( type( testy_L2).__name__ == 'ndarray' )
assert( len(testy_L2) == 62 ) 
assert( np.all( testy_L2[50:60] == [ 0.,  0.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.] )  )
assert( np.all( testy_L2[0:10] == [ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.] ) )

In [78]:
#Followed the same formula for the NN_L2 function 
#but switched the formula 
def L1(a,b):
    dist= np.sum(np.absolute(a-b))
    return np.array(dist)

def NN_L1(xvect, yvect, zvect):
    predict=[]
    for z in zvect:
        distances = [L1(z, xvect[i]) for i in range(len(yvect))]
        index=np.argmin(distances)
        predict.append(yvect[index])
    return np.array(predict)


print("L1 Output: \n", NN_L1(trainx, trainy, testx))
print("L2 Output: \n", NN_L2(trainx, trainy, testx))




L1 Output: 
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 2. 1. 0. 2. 0. 0. 0. 0. 0. 1. 0.]
L2 Output: 
 [0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 2. 0. 2. 0. 0. 0. 0. 0.]


In [79]:
testy_L1 = NN_L1(trainx, trainy, testx)
testy_L2 = NN_L2(trainx, trainy, testx)

assert( type( testy_L1).__name__ == 'ndarray' )
assert( len(testy_L1) == 62 ) 
assert( not all(testy_L1 == testy_L2) )
assert( all(testy_L1[50:60]== [ 0.,  2.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.]) )
assert( all( testy_L1[0:10] == [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.]) )

In [80]:
def error_rate(testy, testy_fit):
    return float(sum(testy!=testy_fit))/len(testy) 

print("Error rate of NN_L1: ", error_rate(testy,testy_L1) )
print("Error rate of NN_L2: ", error_rate(testy,testy_L2) )


Error rate of NN_L1:  0.22580645161290322
Error rate of NN_L2:  0.20967741935483872


In [65]:
print(np.shape(testy))
print(np.shape(testy_L2))

(62,)
(62,)


In [92]:
def confusion(testy, testy_L2):
    #intialized a 3x3 array full of zeros
    matrix = np.zeros((3,3), dtype=int)
    # Cannot be used as indices if it is not an int
    testy=testy.astype(int)
    testy_L2=testy_L2.astype(int)
    for tes, ty in zip(testy, testy_L2):
        # adds 1 to each point in the matrix 
        # at the specific location of [tes,ty]
        matrix[tes, ty] +=1
    return matrix

L2_neo=confusion(testy, testy_L2)
print(type(L2_neo))
print(L2_neo.shape)
print(L2_neo)


<class 'numpy.ndarray'>
(3, 3)
[[17  1  2]
 [10 10  0]
 [ 0  0 22]]


In [95]:
L1_neo = confusion(testy, testy_L1) 
assert( type(L1_neo).__name__ == 'ndarray' )
assert( L1_neo.shape == (3,3) )
assert( np.all(L1_neo == [[ 16.,  2.,  2.],[ 10.,  10.,  0.],[ 0.,  0.,  22.]]) )
L2_neo = confusion(testy, testy_L2)  
assert( np.all(L2_neo == [[ 17.,  1.,  2.],[ 10.,  10.,  0.],[ 0.,  0.,  22.]]) )
print(L1_neo)
print("______________________________________")
print(L2_neo)

[[16  2  2]
 [10 10  0]
 [ 0  0 22]]
______________________________________
[[17  1  2]
 [10 10  0]
 [ 0  0 22]]
